In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install tabulate --user

In [ ]:
from tabulate import tabulate

In [ ]:
#creating the sql database for storing data

In [ ]:
import sqlite3 as sql
# Connect to the SQLite database
mc=sql.connect('/content/drive/MyDrive/bistro.db')
# Create a cursor object to execute SQL queries
cur=mc.cursor()

In [ ]:
cur.execute("drop table if exists Restaurants")
cur.execute("drop table if exists MenuItems")
cur.execute("drop table if exists Orders")
cur.execute("drop table if exists Reservations")
cur.execute("drop table if exists Payments")
cur.execute("drop table if exists Employee")
cur.execute("drop table if exists Resources")

In [ ]:
cur.execute('''CREATE TABLE Restaurants (
    restaurant_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    admin_name VARCHAR(50) NOT NULL,
    address VARCHAR(255) NOT NULL,
    contact INT NOT NULL,
    email_id VARCHAR(200) NOT NULL)
''');

cur.execute('''CREATE TABLE MenuItems (
    menu_id INT AUTO_INCREMENT PRIMARY KEY,
    restaurant_id INT NOT NULL,
    name VARCHAR(255) NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    itemDescription varchar(50),
    itemRating_instars int(5),
    FOREIGN KEY (restaurant_id) REFERENCES Restaurants(restaurant_id))
''')

cur.execute('''CREATE TABLE Orders (
    order_id INT AUTO_INCREMENT PRIMARY KEY,
    menu_id INT NOT NULL,
    quantity INT NOT NULL,
    total_price DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (menu_id) REFERENCES MenuItems(menu_id))
''')

cur.execute('''CREATE TABLE Reservations (
    reservation_id INT AUTO_INCREMENT PRIMARY KEY,
    customer_name VARCHAR(255) NOT NULL,
    reservation_date DATE NOT NULL)
''')

cur.execute('''CREATE TABLE Payments (
    payment_id INT AUTO_INCREMENT PRIMARY KEY,
    order_id INT NOT NULL,
    payment_date TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    amount DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id))
''')

cur.execute('''create table if not exists Employee
            (empId varchar(10) not null primary key,
            empName varchar(30),
            empMobileno int(10),
            empEmail varchar(50),
            empDept varchar(15),
            empSalary int(15),
            empDoj date)''')

cur.execute('''create table if not exists Resources
            (monthName varchar(15),
            resourceName varchar(20) not null,
            totalQuantity_inkgs int(15),
            costperkg_inrs int(15),
            totalCost_inrs int(20),
            quantityUsed_inkgs int(15),
            quantityWasted_inkgs int(15))''')

In [ ]:
#user section

In [ ]:
#Create tables for restaurants, menu items, orders, reservations, and payments
def online_dinein():
  ans1=input("Do you want to check the restaurants? (y/n)")

  while ans1.lower()=='y':
    cur.execute("select * from Restaurants")
    data=cur.fetchall()
    print(data)

    n=int(input("Enter the restaurant which you chose(restaurant_id): "))
    cur.execute("select * from Restaurants where restaurant_id='%s'"%(n))
    data=cur.fetchall()
    print(data)
    cur.execute("select * from MenuItems where restaurant_id='%s'"%(n))
    data=cur.fetchall()
    print(data)

    ans1=input("Do you want to check any other restaurants? ")

  else:
    print("Thank you!!")

def offline_dinein():
  ans1=input("Do you want to check the restaurants? (y/n)")

  while ans1.lower()=='y':
    cur.execute("select * from Restaurants")
    data=cur.fetchall()
    print(data)

    n=int(input("Enter the restaurant which you chose(restaurant_id): "))
    cur.execute("select * from Restaurants where restaurant_id='%s'"%(n))
    data=cur.fetchall()
    print(data)
    cur.execute("select * from MenuItems where restaurant_id='%s'"%(n))
    data=cur.fetchall()
    print(data)

    l=[]
    ans3='y'
    while ans3.lower() =='y':
      m=int(input("Enter the menu id of the selected item: "))
      o=int(input("The quantity to be ordered: "))
      cur.execute("select price from MenuItems where restaurant_id='%s'"%(n))
      data=cur.fetchall()
      final=data*o
      cur.execute("INSERT INTO Orders (menu_id, quantity, total_price) VALUES (%s,%s,%s)"%(m,o,final))
      mc.commit()
      cur.execute("select order_id from Orders where menu_id=%s' quantity=%s, total_price=%s"%(m,o,final))
      fin=cur.fetchall()
      l.append(fin)

      ans3=input("Do you wish to add anymore items?(y/n) ")

    ans1=input("Do you want to check any other restaurants? ")


  else:
    for i in l:
      cur.execute("select * from Orders where order_id=%s"%(i))
      data=cur.fetchall()
      print(data)
    print("Thank you!!")

  ans4=input("Would you like to reserve a table?(y/n) ")
  if ans4 == 'y':
    a=input("Enter the customer name: ")
    b=input("Enter the reservation date(yyyy-mm-dd): ")
    cur.execute("INSERT INTO Reservations (customer_name, reservation_date) VALUES ('%s', '%s')"%(a, b))
    mc.commit()
    cur.execute("select * from Reservations where customer_name='%s'"%(a))

  else:
    print("thank you")

  ans5=input("Would you like to make the payment?(y/n) ")

  if ans5=='y':
    total=0
    for i in l:
      cur.execute("select total_price from Orders where order_id=%s"%(i))
      d=cur.fetchall()
      total+=d
      cur.execute("INSERT INTO Payments (order_id, amount) VALUES (%s, %s)"%(i,d))
      mc.commit()
      cur.execute("select * from Payments where order_id=%s"%(i))
      pay=cur.fetchall()
      print(pay)
      print("Total to be paid: ", total)
  else:
    print("thank you!!!")


In [ ]:
#Admin section

In [ ]:
############### EMPLOYEE ####################
def employee():
  head2=["ID","NAME","MOBILE NO.", "EMAIL ID","DEPARTMENT","SALARY","DOJ"]

  def displayemployees():
    cur.execute("select * from Employee")
    data=cur.fetchall()
    print(tabulate(data, headers=head2, tablefmt="grid"))

  def viewemployees():
    c=int(input('''HOW DO YOU WISH TO VIEW THE EMPLOYEES?
                   1. SINGLE EMPLOYEE USING ID
                   2. SINGLE EMPLOYEE USING NAME
                   3. EMPLOYEES ORDERED BY ID
                   4. EMPLOYEES ORDERED BY NAME
                   5. EMPLOYEES ORDERED BY SALARY
                   6. EMPLOYEES ORDERED BY DATE OF JOINING
                   7. EMPLOYEES IN A DEPARTMENT '''))
    if c==1:
      i=input("\nENTER THE ID OF THE EMPLOYEE YOU WISH TO VIEW: ")
      cur.execute("select * from Employee where empId='%s'"%(i))
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==2:
      n=input("\nENTER THE NAME OF THE EMPLOYEE YOU WISH TO VIEW: ")
      cur.execute("select * from Employee where empName='%s'"%(n))
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==3:
      cur.execute("select * from Employee order by empId")
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==4:
      cur.execute("select * from Employee order by empName")
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==5:
      cur.execute("select * from Employee order by empSalary")
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==6:
      cur.execute("select * from Employee order by empDoj")
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==7:
      d=input("\nENTER THE REQUIRED DEPARTMENT: ")
      cur.execute("select * from Employee where empDept='%s'"%(d))
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    else:
      print("\nINVALID CHOICE")

  def addemployee():
    ch='y'
    while(ch.lower()=='y'):
      i=input("ENTER EMPLOYEE ID: ")
      n=input("ENTER EMPLOYEE NAME: ")
      m=int(input("ENTER EMPLOYEE MOBILE NO.: "))
      e=input("ENTER EMPLOYEE EMAIL ID: ")
      d=input("ENTER EMPLOYEE DEPARTMENT: ")
      s=int(input("ENTER EMPLOYEE SALARY: "))
      doj=input("ENTER EMPLOYEE DATE OF JOINING: ")
      cur.execute("insert into Employee values ('%s','%s',%s,'%s','%s',%s,'%s')"%(i,n,m,e,d,s,doj))
      mc.commit()
      print("\nEMPLOYEE RECORD ADDED SUCCESSFULLY")
      ch=input("\nDO YOU WANT TO ADD MORE RECORDS (y/n)?")

  def updateemployee():
    i=input("ENTER ID FOR THE EMPLOYEE RECORD TO BE UPDATED")
    ch='y'
    while(ch.lower()=='y'):
      c=int(input('''\nCHOOSE THE FIELD TO BE UPDATED:
                       1. NAME
                       2. MOBILE NO.
                       3. EMAIL ID
                       4. DEPARTMENT
                       5. SALARY
                       6. DATE OF JOINING '''))
      if c==1:
        n=input("\nENTER NEW EMPLOYEE NAME: ")
        cur.execute("update Employee set empName = '%s' where empId = '%s'"%(n,i))
        print("\nEMPLOYEE RECORD UPDATED SUCCESSFULLY")
      elif c==2:
        m=int(input("\nENTER NEW EMPLOYEE MOBILE NO.: "))
        cur.execute("update Employee set empMobileno = %s where empId = '%s'"%(m,i))
        print("\nEMPLOYEE RECORD UPDATED SUCCESSFULLY")
      elif c==3:
        e=input("\nENTER NEW EMPLOYEE EMAIL ID: ")
        cur.execute("update Employee set empEmail = '%s' where empId = '%s'"%(e,i))
        print("\nEMPLOYEE RECORD UPDATED SUCCESSFULLY")
      elif c==4:
        d=input("\nENTER NEW EMPLOYEE DEPARTMENT: ")
        cur.execute("update Employee set empDept = '%s' where empId = '%s'"%(d,i))
        print("\nEMPLOYEE RECORD UPDATED SUCCESSFULLY")
      elif c==5:
        s=int(input("\nENTER NEW EMPLOYEE SALARY: "))
        cur.execute("update Employee set empSalary = %s where empId = '%s'"%(s,i))
        print("\nEMPLOYEE RECORD UPDATED SUCCESSFULLY")
      elif c==6:
        doj=input("ENTER NEW EMPLOYEE DATE OF JOINING: ")
        cur.execute("update Employee set empDoj = '%s' where empId = '%s'"%(n,i))
        print("\nEMPLOYEE RECORD UPDATED SUCCESSFULLY")
      else:
        print("\nINVALID CHOICE")
      mc.commit()
      ch=input("\nDO YOU WANT TO MAKE MORE CHANGES (y/n)?")

  def deleteemployee():
    c=int(input('''HOW DO YOU WANT TO DELETE AN EMPLOYEE RECORD?
                   1. USING ID
                   2. USING NAME '''))
    if c==1:
      i=input("ENTER ID FOR THE EMPLOYEE RECORD TO BE DELETED")
      cur.execute("delete from Employee where empId = '%s'"%(i))
      print("\nEMPLOYEE RECORD DELETED SUCCESSFULLY")
    elif c==2:
      n=input("ENTER NAME FOR THE EMPLOYEE RECORD TO BE DELETED")
      cur.execute("delete from Employee where empName = '%s'"%(n))
      print("\nEMPLOYEE RECORD DELETED SUCCESSFULLY")
    else:
      print("\nINVALID CHOICE")

  # main content inside the function
  while True:
      print("\nWELCOME TO THE EMPLOYEE MANAGEMENT PAGE")
      choice=int(input('''\nCHOOSE THE OPERATION TO BE PERFORMED IN THE EMPLOYEE DATABASE:
                            1. DISPLAY EMPLOYEE DATABASE
                            2. VIEW SPECIFIC EMPLOYEE RECORDS
                            3. ADD NEW EMPLOYEE RECORDS
                            4. UPDATE AN EXISTING EMPLOYEE RECORD
                            5. DELETE AN EXISTING EMPLOYEE RECORD
                            6. EXIT PAGE '''))
      if choice==1:
        displayemployees()
      elif choice==2:
        viewemployees()
      elif choice==3:
        addemployee()
      elif choice==4:
        updateemployee()
      elif choice==5:
        deleteemployee()
      elif choice==6:
        break
        return
      else:
        print("\nINVALID CHOICE")


###############  MENU  ##############
def menu_update():
  head1=["ITEM NAME","ITEM DESCRIPTION","ITEM PRICE","ITEM RATING"]

  def displaymenu():
    cur.execute("select * from MenuItems")
    data=cur.fetchall()
    print(tabulate(data, headers=head1, tablefmt="grid"))

  def viewitems():
    c=int(input('''HOW DO YOU WISH TO VIEW THE ITEM
                   1. SINGLE ITEM WITH ITEM NAME
                   2. ITEMS ORDERED BY ITEM NAME  '''))
    if c==1:
      i=input("\nENTER THE ITEM ID FOR THE ITEM YOU WISH TO VIEW: ")
      cur.execute("select * from MenuItems where name='%s'"%(i))
      data=cur.fetchall()
      print(tabulate(data, headers=head1, tablefmt="grid"))
    elif c==2:
      cur.execute("select * from MenuItems order by name")
      data=cur.fetchall()
      print(tabulate(data, headers=head1, tablefmt="grid"))
    else:
      print("\nINVALID CHOICE")

  def additem():
     ch='y'
     i=input("ENTER RESTAURANT ID: ")
     while(ch.lower()=='y'):
      n=input("ENTER ITEM NAME: ")
      d=input("ENTER ITEM DESCRIPTION: ")
      p=int(input("ENTER ITEM PRICE: "))
      r=int(input("ENTER ITEM RATING: "))
      cur.execute("insert into MenuItems(restaurant_id,name,itemDescription,price,itemRating_instars) values ('%s','%s','%s',%s,%s)"%(i,n,d,p,r))
      mc.commit()
      print("\nITEM RECORD SUCCESSFULLY ADDED")
      ch=input("DO YOU WANT TO ADD MORE ITEMS (y/n)? ")

  def updateitem():
    i=input("ENTER ITEM NAME FOR THE ITEM RECORD TO BE UPDATED ")
    ch='y'
    while(ch.lower()=='y'):
      c=int(input('''\nCHOOSE THE FIELD TO BE UPDATED:
                       1. NAME
                       2. DESCRIPTION
                       3. PRICE
                       4. RATING '''))
      if c==1:
        n=input("\nENTER NEW ITEM NAME: ")
        cur.execute("update MenuItems set name = '%s' where name = '%s'"%(n,i))
        print("\nITEM RECORD UPDATED SUCCESSFULLY")
      elif c==2:
        d=input("\nENTER NEW ITEM DESCRIPTION: ")
        cur.execute("update MenuItems set itemDescription = '%s' where name = '%s'"%(d,i))
        print("\nITEM RECORD UPDATED SUCCESSFULLY")
      elif c==3:
        p=int(input("\nENTER NEW ITEM PRICE: "))
        cur.execute("update MenuItems set price = %s where name = '%s'"%(p,i))
        print("\nITEM RECORD UPDATED SUCCESSFULLY")
      elif c==4:
        r=int(input("\nENTER NEW ITEM RATING: "))
        cur.execute("update MenuItems set itemRating_instars = %s where name = '%s'"%(r,i))
        print("\nITEM RECORD UPDATED SUCCESSFULLY")
      else:
        print("\nINVALID CHOICE")
      mc.commit()
      ch=input("\nDO YOU WANT TO MAKE MORE CHANGES (y/n)? ")

  def deleteitem():
    n=input("ENTER ITEM NAME FOR THE ITEM TO BE DELETED")
    cur.execute("delete from MenuItems where itemName = '%s'"%(n))
    print("\nITEM RECORD DELETED SUCCESSFULLY")
    mc.commit()

  # main content inside the function
  while True:
      print("\nWELCOME TO THE MENU MANAGEMENT PAGE")
      choice=int(input('''\nCHOOSE THE OPERATION TO BE PERFORMED IN THE MENU:
                            1. DISPLAY THE MENU
                            2. VIEW SPECIFIC ITEM RECORDS
                            3. ADD NEW ITEM RECORDS
                            4. UPDATE AN EXISTING ITEM RECORD
                            5. DELETE AN EXISTING ITEM RECORD
                            6. EXIT PAGE '''))
      if choice==1:
        displaymenu()
      elif choice==2:
        viewitems()
      elif choice==3:
        additem()
      elif choice==4:
        updateitem()
      elif choice==5:
        deleteitem()
      elif choice==6:
        break
        return
      else:
        print("\nINVALID CHOICE")


######################## STOCKS ####################################
def stocks():
  head2=["MONTH NAME","RESOURCE NAME","TOTAL QUANTITY","COST PER KG","TOTAL COST","QUANTITY USED","QUANTITY WASTED"]

  def displayresources():
    cur.execute("select * from Resources")
    data=cur.fetchall()
    print(tabulate(data, headers=head2, tablefmt="grid"))

  def viewresources():
    c=int(input('''HOW DO YOU WISH TO VIEW THE RESOURCES?
                   1. SINGLE RESOURCE WITH RESOURCE NAME
                   2. ALL RESOURCES IN A MONTH
                   3. RESOURCES ORDERED BY COST PER KG
                   4. RESOURCES ORDERED BY QUANTITY USED
                   5. RESOURCES ORDERED BY QUANTITY WASTED  '''))
    if c==1:
      rn=input("\nENTER THE RESOURCE NAME FOR THE RESOURCE YOU WISH TO VIEW: ")
      cur.execute("select * from Resources where resourceName ='%s'"%(rn))
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==2:
      mn=input("\nENTER THE MONTH NAME WHOSE RESOURCES YOU WISH TO VIEW: ")
      cur.execute("select * from Resources where monthName='%s'"%(mn))
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==3:
      cur.execute("select * from Resources order by costperkg_inrs")
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==4:
      cur.execute("select * from Resources order by quantityUsed_inkgs")
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    elif c==5:
      cur.execute("select * from Resources order by quantityWasted_inkgs ")
      data=cur.fetchall()
      print(tabulate(data, headers=head2, tablefmt="grid"))
    else:
      print("\nINVALID CHOICE")

  def addresource():
    ch='y'
    while(ch.lower()=='y'):
        mn=input("ENTER THE NAME OF THE MONTH: ")
        rn=input("ENTER THE NAME OF THE RESOURCE: ")
        tq=int(input("ENTER THE TOTAL QUANTITY OF RESOURCE PURCHASED (IN KGS): "))
        c=int(input("ENTER THE COST PER KG OF THE RESOURCE (IN RS): "))
        qu=int(input("ENTER THE QUANTITY OF RESOURCE USED (IN KGS): "))
        cur.execute("insert into Resources values ('%s','%s',%s,%s,(%s*%s),%s,(%s-%s))"%(mn,rn,tq,c,tq,c,qu,tq,qu))
        mc.commit()
        print("\nRESOURCE RECORD SUCCESSFULLY ADDED")
        ch=input("DO YOU WANT TO ADD MORE RECORDS (y/n)? ")

  def updateresource():
    rn=input("ENTER THE RESOURE NAME FOR THE RECORD TO BE UPDATED: ")
    mn=input("ENTER THE MONTH NAME FOR THE RESOURCE RECORD TO BE UPDATED: ")
    ch='y'
    while(ch.lower()=='y'):
      c=int(input('''\nCHOOSE THE FIELD TO BE UPDATED:
                       1. COST PER KG
                       2. TOTAL QUANTITY
                       3. QUANTITY USED'''))
      if c==1:
        c=int(input("\nENTER NEW COST PER KG (IN RS): "))
        cur.execute("update Resources set costperkg_inrs = %s where resourceName = '%s' and monthName = '%s'"%(c,rn,mn))
        mc.commit()
        cur.execute("update Resources set totalCost_inrs = totalQuantity_inkgs*costperkg_inrs where resourceName = '%s' and monthName = '%s'"%(rn,mn))
        mc.commit()
        print("\nRESOURCE RECORD UPDATED SUCCESSFULLY")
      elif c==2:
        tq=int(input("\nENTER NEW TOTAL QUANTITY PURCHASED (IN KGS):  "))
        cur.execute("update Resources set totalQuantity_inkgs = %s where resourceName = '%s' and monthName = '%s'"%(tq,rn,mn))
        mc.commit()
        cur.execute("update Resources set totalCost_inrs = totalQuantity_inkgs*costperkg_inrs where resourceName = '%s' and monthName = '%s'"%(rn,mn))
        cur.execute("update Resources set quantityWasted_inkgs = totalQuantity_inkgs - quantityUsed_inkgs where resourceName = '%s' and monthName = '%s'"%(rn,mn))
        mc.commit()
        print("\nRESOURCE RECORD UPDATED SUCCESSFULLY")
      elif c==3:
        qu=int(input("\nENTER NEW QUANTITY USED (IN KGS): "))
        cur.execute("update Resources set quantityUsed_inkgs = %s where resourceName = '%s' and monthName = '%s'"%(qu,rn,mn))
        mc.commit()
        cur.execute("update Resources set quantityWasted_inkgs = totalQuantity_inkgs - quantityUsed_inkgs where resourceName = '%s' and monthName = '%s'"%(rn,mn))
        mc.commit()
        print("\nRESOURCE RECORD UPDATED SUCCESSFULLY")
      else:
        print("\nINVALID CHOICE")
      ch=input("\nDO YOU WANT TO MAKE MORE CHANGES (y/n)? ")

  def calculator():
    mn=input("\nENTER THE NAME OF THE MONTH: ")
    tc_sum=0
    qw_sum=0
    cur.execute("select resourceName, totalCost_inrs from Resources where monthName = '%s'"%(mn))
    data1=cur.fetchall()
    print("\nThe Expenditure in the month of",mn.title(),"is as follows:\n")
    print(tabulate(data1, headers=["RESOURCE","TOTAL COST (IN RS)"], tablefmt="grid"))
    for row in data1:
      tc_sum+=int(row[1])
    print("\nThe Total Expenditure for all the Resources in the month of",mn.title(),"is Rs.",tc_sum)
    cur.execute("select resourceName, quantityWasted_inkgs from Resources where monthName = '%s'"%(mn))
    data2=cur.fetchall()
    print("\nThe Wastage in the month of",mn.title(),"is as follows:\n")
    print(tabulate(data2, headers=["RESOURCE","WASTAGE (IN KGS)"], tablefmt="grid"))
    for row in data2:
      qw_sum+=int(row[1])
    print("\nThe Total Wastage including all the Resources in the month of",mn.title(),"is",qw_sum,"kgs")

  # main content inside the function
  while True:
      print("\nWELCOME TO THE RESOURCES MANAGEMENT PAGE")
      choice=int(input('''\nCHOOSE THE OPERATION TO BE PERFORMED IN THE RESOURCES DATABASE:
                            1. DISPLAY RESOURCES DATABASE
                            2. VIEW SPECIFIC RESOURCE RECORDS
                            3. ADD NEW RESOURCE RECORDS
                            4. UPDATE AN EXISTING RESOURCE RECORD
                            5. MONTHLY EXPENDITURE AND WASTAGE CALCULATION
                            6. EXIT PAGE '''))
      if choice==1:
        displayresources()
      elif choice==2:
        viewresources()
      elif choice==3:
        addresource()
      elif choice==4:
        updateresource()
      elif choice==5:
        calculator()
      elif choice==6:
        break
        return
      else:
        print("\nINVALID CHOICE")

######################## ORDERS AND TABLES ######################
def orders_and_table():
  i=int(input('''What do you wish to check
  1. Order details
  2. Table details'''))

  if i==1:
    que=int(input('''How do you wish to check the data?
    1. By Order id
    2. Check all the orders''' ))

    if que==1:
      j=int(input("Enter the order id: "))
      cur.execute("select * from Orders where order_id=%s"%(j))
      data=cur.fetchall()
      for n in data:
        print(data)

    elif que==2:
      cur.execute("select * from Orders")
      data=cur.fetchall()
      for n in data:
        print(data)

    else:
      print("Invalid entry")

  elif i==2:
    que=int(input('''How do you wish to check the data?
    1. By Resevation id
    2. By Customer name
    2. Check all the orders''' ))

    if que==1:
      j=int(input("Enter the order id: "))
      cur.execute("select * from Reservations where reservation_id=%s"%(j))
      data=cur.fetchall()
      for n in data:
        print(data)

    elif que==2:
      cur.execute("select * from Reservations where customer_name=%s"%(j))
      data=cur.fetchall()
      for n in data:
        print(data)

    elif que==3:
      cur.execute("select * from Reservations")
      data=cur.fetchall()
      for n in data:
        print(data)

    else:
      print("Invalid entry")


#################### FINANCE ###########################
def finance():
  i=int(input('''How do you wish to check the bills:
  1. Payment id
  2. Entire payment history'''))

  if i==1:
    j=int(input("Enter the payment id: "))
    cur.execute("select * from Payments where payment_id=%s"%(j))
    data=cur.fetchall()
    for n in data:
      print(data)

  elif i==2:
    cur.execute("select * from Reservations")
    data=cur.fetchall()
    for n in data:
      print(data)

  else:
    print("Invalid entry")

In [ ]:
#Main code

In [ ]:
print("\nWELCOME TO LOGIN PAGE!")
while True:
  ans = int(input('''HOW WOULD YOU LOGIN AS
  1. USER
  2. ADMIN
  3. OUTLET'''))
  user_database = {}
  if ans == 1:
    # Sample user database (dictionary for simplicity)
    def sign_up(username, password):
      if username in user_database:
        return "Username already exists. Please choose another."
      else:
        user_database[username] = password
        return "Sign-up successful. You can now log in."
    def login(username, password):
      if username in user_database and user_database[username] == password:
        return "Login successful. Welcome, " + username + "!"
      while True:
        print("\nWELCOME TO DASHBOARD!")
        choice=int(input('''CHOOSE THE OPARATION YOU WISH TO PERFORM
                                1. ONLINE DINE IN
                                2. OFFLINE DINE IN
                                3. EXIT ''' ))

        if choice==1:
          online_dinein()
        elif choice==2:
          offline_dinein()
        elif choice==3:
          print("\nLOGGING YOU OUT...")
          break
        else:
            print("\nINVALID CHOICE")
            break
      else:
        return "Login failed. Invalid username or password."

    while True:
      print("1. Sign Up")
      print("2. Login")
      print("3. Quit")
      choice = input("Enter your choice (1/2/3): ")
      if choice == '1':
        username = input("Enter a username: ")
        password = input("Enter a password: ")
        result = sign_up(username, password)
        print(result)
      elif choice == '2':
        username = input("Enter your username: ")
        password = input("Enter your password: ")
        result = login(username, password)
        print(result)
      elif choice == '3':
        break
      else:
        print("Invalid choice. Please try again.")

  elif ans==2:
    def sign_up(username, password):
      if username in user_database:
        return "Username already exists. Please choose another."
      else:
        user_database[username] = password
        return "Sign-up successful. You can now log in."
    def login(username, password):
      if username in user_database and user_database[username] == password:
         print("Login successful. Welcome, " + username + "!")
         while True:
          print("\nWELCOME TO DASHBOARD!")
          choice=int(input('''CHOOSE THE OPARATION YOU WISH TO PERFORM
                                1. EMPLOYEE
                                2. FINANCE
                                3. MENU UPDATE
                                4. ORDERS AND TABLE
                                5. STOCKS
                                6. EXIT ''' ))

          if choice==1:
            employee()
          elif choice==2:
            finance()
          elif choice==3:
            menu_update()
          elif choice==4:
            orders_and_table()
          elif choice==5:
            stocks()
          elif choice==6:
            print("\nLOGGING YOU OUT...")
            break
          else:
            print("\nINVALID CHOICE")
            break
      else:
        return "Login failed. Invalid username or password."

    while True:
      print("1. Sign Up")
      print("2. Login")
      print("3. Quit")
      choice = input("Enter your choice (1/2/3): ")
      if choice == '1':
        username = input("Enter a username: ")
        password = input("Enter a password: ")
        result = sign_up(username, password)
        print(result)
      elif choice == '2':
        username = input("Enter your username: ")
        password = input("Enter your password: ")
        result = login(username, password)
        print(result)
      elif choice == '3':
        break
      else:
        print("Invalid choice. Please try again.")

  elif ans == 3:
    i=int(input('''Do you wish to
    1. Register
    2. Discount'''))

    if ans==1:
      restaurant_name=input("Enter the Restaurant name: ")
      username = input("Enter your username: ")
      add=input("Enter the address: ")
      contact=int(input("Enter contact number: "))
      email_id=input("Enter the email id: ")
      cur.execute("INSERT INTO Restaurants (name,admin_name,address,contact,email_id) VALUES ('%s','%s','%s',%s,'%s')"%(restaurant_name,username,add,contact,email_id))
      mc.commit()
      password = input("Enter your password: ")
      user_database[username] = password

  else:
    print("Invalid input")



WELCOME TO LOGIN PAGE!
